In [10]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ximo/GitHub/skforecast


In [11]:
import re
import pytest
import numpy as np
import pandas as pd
from skforecast.direct import ForecasterDirect
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

# Fixtures
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog
from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog_predict
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [12]:
forecaster = ForecasterDirect(
    LinearRegression(), steps=5, lags=3, differentiation=1
)
forecaster.fit(y=y, store_in_sample_residuals=True)
results = forecaster._predict_interval_conformal(
    steps=5, nominal_coverage=0.95, use_in_sample_residuals=True, use_binned_residuals=False
)

expected = pd.DataFrame(
                data    = np.array(
                            [[0.65659084,  0.17710854,  1.13607314],
                                [0.6496844 , -0.40541691,  1.7047857 ],
                                [0.61632235, -0.98830203,  2.22094673],
                                [0.59545137, -1.53828123,  2.72918398],
                                [0.64917089, -2.01927988,  3.31762167]]
                        ),
                columns = ['pred', 'lower_bound', 'upper_bound'],
                index   = pd.RangeIndex(start=50, stop=55, step=1)
            )
results.to_numpy()

array([[ 0.65659084,  0.11145843,  1.20172325],
       [ 0.6496844 , -0.44058042,  1.73994922],
       [ 0.61632235, -1.01907488,  2.25171958],
       [ 0.59545137, -1.58507827,  2.77598101],
       [ 0.64917089, -2.07649116,  3.37483294]])

In [38]:
forecaster = ForecasterDirect(LinearRegression(), steps=2, lags=3)
forecaster.fit(y=pd.Series(np.arange(10)), store_in_sample_residuals=True)
forecaster.in_sample_residuals_ = {
    1: np.full_like(forecaster.in_sample_residuals_[1], fill_value=10),
    2: np.full_like(forecaster.in_sample_residuals_[2], fill_value=20)
}
forecaster.in_sample_residuals_ 

{1: array(10.), 2: array(20.)}

In [23]:
def test_predict_dist_output_when_forecaster_is_LinearRegression_steps_is_2_in_sample_residuals_False_exog_and_transformer():
    """
    Test output of predict_dist when regressor is LinearRegression,
    2 steps are predicted, using out-sample residuals, exog is included and both
    inputs are transformed.
    """
    forecaster = ForecasterDirect(
                     regressor        = LinearRegression(),
                     steps            = 2,
                     lags             = 3,
                     transformer_y    = StandardScaler(),
                     transformer_exog = StandardScaler(),
                 )
    
    forecaster.fit(y=y, exog=exog, store_in_sample_residuals=True)
    forecaster.out_sample_residuals_ = forecaster.in_sample_residuals_
    results = forecaster.predict_dist(
                  steps                   = 2,
                  exog                    = exog_predict,
                  distribution            = norm,
                  n_boot                  = 4,
                  use_in_sample_residuals = False,
                  use_binned_residuals    = False
              )
    expected = pd.DataFrame(
                   data    = np.array([[0.54274594, 0.20807726],
                                        [0.32046382, 0.13511556]]),
                   columns = ['loc', 'scale'],
                   index   = pd.RangeIndex(start=50, stop=52)
               )

    pd.testing.assert_frame_equal(expected, results)

test_predict_dist_output_when_forecaster_is_LinearRegression_steps_is_2_in_sample_residuals_False_exog_and_transformer()

AssertionError: DataFrame.iloc[:, 0] (column name="loc") are different

DataFrame.iloc[:, 0] (column name="loc") values are different (100.0 %)
[index]: [50, 51]
[left]:  [0.54274594, 0.32046382]
[right]: [0.735401729117992, 0.3290149298800433]
At positional index 0, first diff: 0.54274594 != 0.735401729117992